# Annual outlet glacier terminus positions in northwestern Greenland


In [ ]:
# Import modules
import sys
sys.path.append('/mnt/e/')

import os
import geopandas as gpd
import pandas as pd
import GlacierClass as gc
import metrics as met
import plotlib as gpl
import climate as clm
import climplotlib as cpl
import utilities as utils
import plotutils as plu
import matplotlib.pyplot as plt
from shapely.ops import Point
import cartopy.crs as ccrs
import numpy as np
import pwlf
import matplotlib.ticker as mticker
import rasterio as rio
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
# User Parameters

# -- Glacier geodatabase
# ---- relative path for file geodatabase container glacier information
fgdb = '/mnt/e/northwest-decadal/gis-data/nwgreenland_decadal.gdb'
# ---- file geodatabase layer names
termini_layer = 'annual_termini'
points_layer = 'glacier_points'
boxes_layer = 'glacier_boxes'

# -- Climate directories and variables
ecco_sst_dir = '/mnt/e/data/ECCO-V5alpha/'
icesdk_file = '/mnt/e/data/ices_dk_NWGreenland_ocntemp/1118187b.csv'
hadley_sst_dir = '/mnt/e/data/hadley-oi/SST/'
ecco_sic_dir = '/mnt/e/data/ECCO_v5alpha/SIarea/'
hadley_sic_dir = '/mnt/e/data/hadley-oi/ICE/'
noaa_sic_dir = '/mnt/e/data/NSIDC_NOAA_G02202V3_monthly_sic/'
noaa_var = 'goddard_merged_seaice_conc_monthly'
mar_dir = '/mnt/e/data/MARv3.11/ERA_1979-2020-6km/'
fglacier_points = '/mnt/e/northwest-decadal/glacier_points.csv'

# -- Other data files
oceanmask_file = '/mnt/e/data/GimpOceanMask_90m/GimpOceanMask_90m.shp'
bedmachine_file = '/mnt/e/data/BedMachineGreenland_v3/BedMachineGreenland_Bed250m.tif'
measures_file = '/mnt/e/data/MEaSUREs_Greenland_20yr_velocity_v1/greenland_vel_mosaic_v_v1_500m.tif'

# -- Relative path to save figure output
outpath = '/mnt/e/northwest-decadal/manuscript/'

# -- Spatial bounds
ocean_bounds = [82, 68, -40, -76]
ice_bounds = [79, 68, -47, -74]
ocean_depth = 250
topo_range = [0, 1000]

# -- Temporal bounds
times = ['1972-09-01', '2020-08-31']

# -- Thresholds
sic_minimum = 15
ice_minimum = 90

# -- Discrete sampling points
ocean_points = [(69.25, -52.25),
                (70.85, -54.25),
                (73.25, -58.00),#(73.00, -57.50),
                (74.50, -60.00),#(74.25, -58.25),
                (75.50, -61.75),
                (75.75, -65.00),
                (76.25, -71.00),
                (77.00, -72.50)]#(77.25, -72.75)]
ocean_point_names = ['Disko Bay',
                     'Uummannaq Fjord',
                     'Upernavik Icefjord',
                     'South Melville Bay',
                     'Central Melville Bay',
                     'North Melville Bay',
                     'Wolstenholme Bay',
                     'Inglefield Fjord']
glacier_points = pd.read_csv(fglacier_points)
glacier_coords = [(glacier_points.iloc[g].Latitude, glacier_points.iloc[g].Longitude) for g in glacier_points.index]

# -- Geographic information (for plotting)
projection = ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70)

# -- Select figure formatting style and set default figure properties
style = 'pub-cryo'
figformat = 'pdf'
figwidth = plu.figureWidth(style)
plu.globalDesignProperties(style)

In [ ]:
# GLACIER DATA ANALYSIS

# -- Load glacier data
termini = gpd.read_file(fgdb, layer=termini_layer, driver='FileGDB')

points = gpd.read_file(fgdb, layer=points_layer, driver='FileGDB')
points.sort_values(by='Glacier_ID', inplace=True)
points.set_index('Glacier_ID', drop=False, inplace=True)

refboxes = gpd.read_file(fgdb, layer=boxes_layer, driver='FileGDB')
refboxes.sort_values(by='Glacier_ID', inplace=True)
refboxes.set_index('Glacier_ID', drop=False, inplace=True)

# -- Get scope of glaciers and time
GIDS = points.Glacier_ID.values

termini['Year'] = pd.to_datetime(termini['Year'], format='%Y')
YEAR_START = termini.Year.min().year
YEAR_END = termini.Year.max().year
YEARS = range(YEAR_START, YEAR_END)

DATE_START = termini.Source_Date.min()
DATE_END = termini.Source_Date.max()

# -- Initialize dictionary of Glacier objects to store glacier information
all_glaciers = {id: gc.Glacier(id) for id in GIDS}
for id in all_glaciers:
    all_glaciers[id].refbox = refboxes.loc[id].geometry
    all_glaciers[id].officialname = points.loc[id].Official_Name
    all_glaciers[id].greenlandicname = points.loc[id].Greenlandic_Name
    all_glaciers[id].alternativename = points.loc[id].Alternative_Name

# -- Construct an observation time series for each glacier
for id in GIDS:

    # get reference line and all observations for glacier ID
    glacier = termini.query('Glacier_ID == @id')
    # refbox = all_glaciers[id].refbox

    # loop through all observations and process data
    for n in range(len(glacier)):
        observation = glacier.iloc[n]

        # create a Terminus Observation for a row in geodataframe
        obs = gc.TerminusObservation(gid=int(observation.Glacier_ID),
                                     qflag=observation.Quality_Flag,
                                     termination='',
                                     imageid=observation.Image_ID,
                                     sensor=observation.Sensor,
                                     date=pd.to_datetime(
                                         observation.Source_Date),
                                     terminus=observation.geometry,
                                     referencebox=all_glaciers[id].refbox)

        # add glacier observation to time series
        all_glaciers[id].addObservation(obs)
    
    # update all attribute series for glacier
    all_glaciers[id].updateObservedValues()
    all_glaciers[id].updateDerivedValues(YEARS)

# Figure 1: Northwest Greenland study area

In [ ]:
# -- Load GIMP ocean mask
gimp = gpd.read_file(oceanmask_file)
gimp.to_crs(points.crs)

# -- 0: Common features between both plots (function)
def plot_common_map_features(ax, gimp, points, ocean_points):
    # -- Set map boundaries based on observed region + some buffer
    bounds = points.total_bounds
    bound_buffer = 100000
    ax.set_xlim([bounds[0]-bound_buffer, bounds[2]+bound_buffer])
    ax.set_ylim([bounds[1]-bound_buffer, bounds[3]+bound_buffer])
    # -- Set gridline properties to display as lat/lon and below other features
    gl = ax.gridlines(
        transform=ccrs.Geodetic(), 
        draw_labels=True,
        x_inline=False,
        y_inline=False,
        color='dimgray',
        zorder=0.5)
    gl.xlabels_top = True
    gl.xlabels_bottom = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.xlocator = mticker.FixedLocator([-70, -60, -50])
    # -- Convert ocean points to geographic data
    xy_ocean_points = [(c[1], c[0]) for c in ocean_points]
    ocean_zones = gpd.GeoDataFrame({'geometry': [Point(c) for c in xy_ocean_points]}, crs='EPSG:4326')
    ocean_zones = ocean_zones.to_crs(points.crs)
    # -- Plot Greenland outline, glacier points, and ocean points
    gimp.plot(ax=ax, linewidth=0.25, color='dimgray')
    points.plot(ax=ax, markersize=15, color='white', edgecolor='black', linewidth=0.75, zorder=2, label='Glaciers')
    ocean_zones.plot(ax=ax, markersize=40, color='darkorange', edgecolor='black', linewidth=0.75, label='Ocean data points')
    for x, y, label in zip(ocean_zones.geometry.x, ocean_zones.geometry.y, ocean_zones.index.values+1):
        ax.annotate(label, xy=(x, y), xytext=(-12,-2), textcoords='offset points', color='darkorange', fontweight='bold')

# -- Adjust colorbar size to match plot
def cbar_adjust(mappable, extend='neither'):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('bottom', size='5%', pad=0.05, axes_class=plt.Axes)
    cbar = fig.colorbar(mappable, cax=cax, orientation='horizontal', extend=extend)
    return cbar

# -- Initialize figure
fig, axs = plt.subplots(
    nrows=1, ncols=2,
    figsize=(2*figwidth, 2*figwidth), 
    subplot_kw={'projection': projection})
axs = axs.flatten()

# -- 1: BEDMACHINE BASEMAP
axs[0].annotate(text='(a)', xy=(-0.15, 1.05), xycoords='axes fraction')
plot_common_map_features(axs[0], gimp, points, ocean_points)
bedmachine = rio.open(bedmachine_file)
xmin, ymin, xmax, ymax = bedmachine.bounds
extent = [xmin, xmax, ymin, ymax]
bm = axs[0].imshow(bedmachine.read(1), cmap='gist_earth', vmin=-1000, vmax=1000, extent=extent, transform=projection)
bm_cbar = cbar_adjust(bm, extend='both')
bm_cbar.set_label('Bed topography (m)')
# -- Annotate glacier points
for x, y, label in zip(points.geometry.x[9::10], points.geometry.y[9::10], points.Glacier_ID[9::10]):
    axs[0].annotate(label, xy=(x, y), xytext=(10, 10), textcoords='offset points', color='white', fontweight='bold',
    arrowprops={'arrowstyle': '->', 'shrinkA': .1, 'color': 'white'})

# -- 2: GIMP VELOCITY BASEMAP
axs[1].annotate(text='(b)', xy=(-0.15, 1.05), xycoords='axes fraction')
plot_common_map_features(axs[1], gimp, points, ocean_points)
measures_data = rio.open(measures_file)
measures = measures_data.read(1, masked=True)
xmin, ymin, xmax, ymax = measures_data.bounds
extent = [xmin, xmax, ymin, ymax]
cm_hsv = colors.LinearSegmentedColormap.from_list(name='cm_hsv', colors=cm.hsv(np.linspace(0.1, 1, 100)))
isv = axs[1].imshow(measures, cmap=cm_hsv, norm=colors.LogNorm(vmin=1, vmax=3000), extent=extent, transform=projection)
isv_cbar = cbar_adjust(isv, extend='both')
isv_cbar.set_label('Ice sheet velocity (m/yr)')
# -- Annotate named glaciers
id_list_a = [12, 17, 26, 42, 55, 66, 81, 87]
name_list_a = ['KA', 'RI', '26', 'KJ', '55', '66', 'TR', 'MJ']
for x, y, label in zip(points.geometry.x[id_list_a], points.geometry.y[id_list_a], name_list_a):
    axs[1].annotate(label, xy=(x, y), xytext=(10, 10), textcoords='offset points', color='white', fontweight='bold',
    arrowprops={'arrowstyle': '->', 'shrinkA': .1, 'color': 'white'})
id_list_b = [3, 8, 9, 11, 16, 25, 36, 51, 86]
name_list_b = ['JI', 'SA', 'ST', 'SI', 'KS', 'NS', '36', 'KO', 'VH']
for x, y, label in zip(points.geometry.x[id_list_b], points.geometry.y[id_list_b], name_list_b):
    axs[1].annotate(label, xy=(x, y), xytext=(10, 0), textcoords='offset points', color='white', fontweight='bold',
    arrowprops={'arrowstyle': '->', 'shrinkA': .1, 'color': 'white'})
id_list_c = [7, 21, 35]
name_list_c = ['AS', 'UI', 'AL']
for x, y, label in zip(points.geometry.x[id_list_c], points.geometry.y[id_list_c], name_list_c):
    axs[1].annotate(label, xy=(x, y), xytext=(10, -10), textcoords='offset points', color='white', fontweight='bold',
    arrowprops={'arrowstyle': '->', 'shrinkA': .1, 'color': 'white'})


fig.legend(handles=axs[0].get_legend_handles_labels()[0][:2], ncol=2, loc='upper center', bbox_to_anchor=(0.5, 0))

plt.tight_layout()

plt.savefig('{}fig1_studyarea.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}fig1_studyarea.png'.format(outpath), bbox_inches='tight', dpi=300)

In [ ]:
del(gimp)
del(bedmachine)
del(measures)

# Figure 2: Terminus records
Annual observations (filled circle) and interpolations (empty circle) for each glacier. Red line indicates first year in which all glaciers have either an observed or interpolated measurement.

In [ ]:
fig, ax = plt.subplots(figsize=(figwidth*2, figwidth*1.1))
gpl.annualObservations(ax, all_glaciers, YEARS, show_firstyear=False, style=style)

plt.savefig('{}fig2_observations.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}fig2_observations.png'.format(outpath), bbox_inches='tight', dpi=300)


# Figure 3: Box method example
This figure was created in ArcGIS Pro.

# Results: terminus behavior

In [ ]:
# -- Identify individual glacier behaviors and calculate basic stats

first_full_year = pd.to_datetime(str(met.firstFullYear(all_glaciers))+'-09')

dominant_length = met.dominantGlaciers(all_glaciers, 'interplengths', \
    startdate=first_full_year)
dominant_area = met.dominantGlaciers(all_glaciers, 'interpareas', \
    startdate=first_full_year)
stable_length = met.stableGlaciers(all_glaciers, 'interplengths', \
    startdate=first_full_year)
stable_area = met.stableGlaciers(all_glaciers, 'interpareas', \
    startdate=first_full_year)
advancing = met.advancingGlaciers(
    all_glaciers, startdate=first_full_year).sort_values().head()
growing = met.growingGlaciers(
    all_glaciers, startdate=first_full_year).sort_values().head()
retreating = met.retreatingGlaciers(
    all_glaciers, startdate=first_full_year).sort_values().head()
shrinking = met.shrinkingGlaciers(
    all_glaciers, startdate=first_full_year).sort_values().head()

glaciers_nondom_length = met.filterGlaciers(all_glaciers, \
    dominant_length.index.values, idtype='remove')
glaciers_nondom_area = met.filterGlaciers(all_glaciers, \
    dominant_area.index.values, idtype='remove')
glaciers_dom_length = met.filterGlaciers(all_glaciers, \
    dominant_length.index.values, idtype='keep')
glaciers_dom_area = met.filterGlaciers(all_glaciers, \
    dominant_area.index.values, idtype='keep')

gid_names_dom_length = [(all_glaciers[g].gid, all_glaciers[g].name) for g in dominant_length.keys()]
gid_names_dom_area = [(all_glaciers[g].gid, all_glaciers[g].name) for g in dominant_area.keys()]
gid_names_stable_length = [(all_glaciers[g].gid, all_glaciers[g].name) for g in stable_length.keys()]
gid_names_stable_area = [(all_glaciers[g].gid, all_glaciers[g].name) for g in stable_area.keys()]
gid_names_advancing = [(all_glaciers[g].gid, all_glaciers[g].name) for g in advancing.keys()]
gid_names_growing = [(all_glaciers[g].gid, all_glaciers[g].name) for g in growing.keys()]
gid_names_retreating = [(all_glaciers[g].gid, all_glaciers[g].name) for g in retreating.keys()]
gid_names_shrinking = [(all_glaciers[g].gid, all_glaciers[g].name) for g in shrinking.keys()]

cumul_area_change = met.finalNetChange(all_glaciers, 'interpareas', startdate=first_full_year).sum()
cumul_length_change = met.finalNetChange(all_glaciers, 'interplengths', startdate=first_full_year).sum()

mean_nondom_area_change = met.finalNetChange(glaciers_nondom_area, 'interpareas', startdate=first_full_year).mean()
mean_nondom_length_change = met.finalNetChange(glaciers_nondom_length, 'interplengths', startdate=first_full_year).mean()

In [ ]:
print("Number of terminus positions traced: ", len(termini))
print('Number of images used: ', len(termini.Image_ID.unique()))
print("Number of glaciers: ", len(termini.Glacier_ID.unique()))
print("First date observed: ", termini.Source_Date.min())
print("Last date observed: ", termini.Source_Date.max())
print("Median number of observations per glacier: {:.0f}".format(len(termini)/len(termini.Glacier_ID.unique())))
print("First year of full observations/interpolations: ", first_full_year)
print('Cumulative area change of all glaciers: {:.0f} km2'.format(cumul_area_change))
print('Cumulative length change of all glaciers: {:.0f} km'.format(cumul_length_change))
print('Mean area change of non-dominant glaciers: {:.1f} km2'.format(mean_nondom_area_change))
print('Mean length change of non-dominant glaciers: {:.1f} km'.format(mean_nondom_length_change))

print('\nDominant length glaciers:\n', dominant_length)
print('\n'.join(str(gn) for gn in gid_names_dom_length))
print('\nDominant area glaciers:\n', dominant_area)
print('\n'.join(str(gn) for gn in gid_names_dom_area))
print('\nStable length glaciers:\n', stable_length)
print('\n'.join(str(gn) for gn in gid_names_stable_length))
print('\nStable area glaciers:\n', stable_area)
print('\n'.join(str(gn) for gn in gid_names_stable_area))
print('\nAdvancing glaciers:\n', advancing)
print('\n'.join(str(gn) for gn in gid_names_advancing))
print('\nGrowing glaciers:\n', growing)
print('\n'.join(str(gn) for gn in gid_names_growing))


# Individual Glaciers
Print length and area information about an individual glacier, based on the glacier ID number.

In [ ]:
id = 3

length_type = met.lengthType(all_glaciers[id], startdate=first_full_year)
area_type = met.areaType(all_glaciers[id], startdate=first_full_year)
cumul_length_change = all_glaciers[id].cumulativeChange('lengths', startdate=first_full_year)
cumul_area_change = all_glaciers[id].cumulativeChange('areas', startdate=first_full_year)

print('\nGlacier: #{} ({})'.format(id, all_glaciers[id].name))
print('\nLength type: {}'.format(length_type))
print('Cumulative length change since {}: {:.1f} km'.format(first_full_year, cumul_length_change[0].iloc[-1]))
print('Length variability: {:.2f}'.format(cumul_length_change[0].std()))
print('\nArea type: {}'.format(area_type))
print('Cumulative area change since {}: {:.1f} km2'.format(first_full_year, cumul_area_change[0].iloc[-1]))
print('Area variability: {:.2f}'.format(cumul_area_change[0].std()))

fig, ax = plt.subplots(figsize=(7,5))
gpl.cumulativeChange(ax, all_glaciers[id], 'areas', startdate=first_full_year)
ax.set_xticks(['1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
ax.set_xticklabels(['1980', '1990', '2000', '2010', '2020'])

axy = ax.twinx()
gpl.cumulativeChange(axy, all_glaciers[id], 'lengths', startdate=first_full_year)
axy.title.set_visible(False)
axy.set_ylabel('Equivalent length (km)')
axy.grid(False)

# Figure 4: Summarized glacier area and length changes
Dominant glacier area (a) and length (b) changes and non-dominant glacier area (c) and length (d).

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(figwidth*2, figwidth*2.5)) 
axs = axs.flatten()

# -- Area change - dominant glaciers
gpl.changeSummary(axs[0], glaciers_dom_area, 'areas', GIDS, startdate=first_full_year)
axs[0].annotate(text='(a)', xy=(-0.15, 1.05), xycoords='axes fraction')
axs[0].set_xlim(left='1975-01-01', right='2022-01-01')
axs[0].set_xticks(['1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
axs[0].set_xticklabels(['1980', '1990', '2000', '2010', '2020'])
axs[0].legend().remove()
axs[0].annotate('JI', xy=('2010-01-01', -125), xytext=(-10, -10), textcoords='offset points',
    arrowprops={'arrowstyle': '-', 'shrinkA': .5})
axs[0].annotate('AL', xy=('2002-01-01', -5), xytext=(5, 5), textcoords='offset points',
    arrowprops={'arrowstyle': '-', 'shrinkA': .5})
axs[0].annotate('KJ', xy=('2010-01-01', -25), xytext=(5, 5), textcoords='offset points',
    arrowprops={'arrowstyle': '-', 'shrinkA': .5})
axs[0].annotate('TR', xy=('1990-01-01', -20), xytext=(-10, -10), textcoords='offset points',
    arrowprops={'arrowstyle': '-', 'shrinkA': .5})

# -- Length change - dominant glaciers
gpl.changeSummary(axs[1], glaciers_dom_length, 'lengths', GIDS, startdate=first_full_year)
axs[1].annotate(text='(b)', xy=(-0.15, 1.05), xycoords='axes fraction')
axs[1].set_xlim(left='1975-01-01', right='2022-01-01')
axs[1].set_xticks(['1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
axs[1].set_xticklabels(['1980', '1990', '2000', '2010', '2020'])
axs[1].set_ylabel('Equivalent length (km)')
axs[1].legend().remove()
axs[1].annotate('JI', xy=('2010-01-01', -16), xytext=(-10, -10), textcoords='offset points',
    arrowprops={'arrowstyle': '-', 'shrinkA': .5})
axs[1].annotate('AL', xy=('2002-01-01', -1), xytext=(5, 5), textcoords='offset points',
    arrowprops={'arrowstyle': '-', 'shrinkA': .5})
axs[1].annotate('36', xy=('1986-01-01', -3), xytext=(2, 2), textcoords='offset points',
    arrowprops={'arrowstyle': '-', 'shrinkA': .5})
axs[1].annotate('KJ', xy=('2012-01-01', -6), xytext=(5, 5), textcoords='offset points',
    arrowprops={'arrowstyle': '-', 'shrinkA': .5})
axs[1].annotate('TR', xy=('1988-01-01', -5), xytext=(-10, -10), textcoords='offset points',
    arrowprops={'arrowstyle': '-', 'shrinkA': .5})

# -- Area change - nondominant glaciers
gpl.changeSummary(axs[2], glaciers_nondom_area, 'areas', startdate=first_full_year)
axs[2].annotate(text='(c)', xy=(-0.15, 1.05), xycoords='axes fraction')
axs[2].set_xlim(left='1975-01-01', right='2022-01-01')
axs[2].set_xticks(['1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
axs[2].set_xticklabels(['1980', '1990', '2000', '2010', '2020'])

# -- Length change - nondominant glaciers
gpl.changeSummary(axs[3], glaciers_nondom_length, 'lengths', startdate=first_full_year)
axs[3].annotate(text='(d)', xy=(-0.15, 1.05), xycoords='axes fraction')
axs[3].set_xlim(left='1975-01-01', right='2022-01-01')
axs[3].set_xticks(['1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
axs[3].set_xticklabels(['1980', '1990', '2000', '2010', '2020'])
axs[3].set_ylabel('Equivalent length (km)')

# -- Final area change - all glaciers
final_area_changes = np.zeros(len(GIDS))
for n in np.arange(len(GIDS)):
    gid = GIDS[n]
    final_area_change = all_glaciers[gid].cumulativeChange('areas', startdate=first_full_year)[0].iloc[-1]
    final_area_changes[n] = final_area_change
axs[4].annotate(text='(e)', xy=(-0.15, 1.05), xycoords='axes fraction')
axs[4].hist(final_area_changes, bins=np.arange(-150,20,10), rwidth=0.8, color='royalblue')
axs[4].set_title('Net area change')
axs[4].set_xlabel('Area change (km$^2$)')
axs[4].set_ylabel('Count')

# -- Final length change - all glaciers
final_length_changes = np.zeros(len(GIDS))
for n in np.arange(len(GIDS)):
    gid = GIDS[n]
    final_length_change = all_glaciers[gid].cumulativeChange('lengths', startdate=first_full_year)[0].iloc[-1]
    final_length_changes[n] = final_length_change
axs[5].annotate(text='(f)', xy=(-0.15, 1.05), xycoords='axes fraction')
axs[5].hist(final_length_changes, bins=np.arange(-20,5,2.5), rwidth=0.8, color='royalblue')
axs[5].set_title('Net length change')
axs[5].set_xlabel('Length change (km)')
axs[5].set_ylabel('Count')

plt.tight_layout()

plt.savefig('{}fig4_changes.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}fig4_changes.png'.format(outpath), bbox_inches='tight', dpi=300)


# Figure 5:
Normalized glacier area (a) and length (b) changes shown in gray, with mean and standard deviation overlain in blue. Histograms of the years in which change points occur in area (c) and length (d) trends for each glacier.

In [ ]:
n_segs = 2

fig, axs = plt.subplots(nrows=1, ncols=2, sharex=True, figsize=(figwidth*2, figwidth))
axs = axs.flatten()

# -- Normalized area (and length) change
gpl.changeSummaryNorm(axs[0], all_glaciers, 'interpareas', startdate=first_full_year, showmean=True)
axs[0].annotate(text='(a)', xy=(-0.15, 1.05), xycoords='axes fraction')
axs[0].set_xlim(left='1975-01-01', right='2022-01-01')
axs[0].set_xticks(['1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
axs[0].set_xticklabels(['1980', '1990', '2000', '2010', '2020'])
axs[0].set_title('Normalized cumulative size change')
axs[0].set_xlabel('Hydrological year')
# axs[0].legend().remove()

# -- Area (and length) change breakpoints
gpl.breakPointsHist(axs[1], all_glaciers, 'interpareas', time_bins=YEARS, n_segs=n_segs, startdate=first_full_year)
axs[1].annotate(text='(b)', xy=(-0.15, 1.05), xycoords='axes fraction')
axs[1].set_xlim(left='1975-01-01', right='2022-01-01')
axs[1].set_xticks(['1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
axs[1].set_xticklabels(['1980', '1990', '2000', '2010', '2020'])
axs[1].set_title('Break point years')
axs[1].set_xlabel('Hydrological year')

# fig.legend(handles=axs[0].get_legend_handles_labels()[0][-3:], loc='lower left', bbox_to_anchor=(0.25, -0.25))

plt.tight_layout()

plt.savefig('{}fig5_breakpoints.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}fig5_breakpoints.png'.format(outpath), bbox_inches='tight', dpi=300)


In [ ]:
n_segs = 2

norm_mean_area, norm_std_area = met.normChangeStats(all_glaciers, 'interpareas', startdate=first_full_year)
norm_mean_area = norm_mean_area[~np.isnan(norm_mean_area)]
norm_mean_area_dates = norm_mean_area.index.values
pwlf_fun_area = pwlf.PiecewiseLinFit(norm_mean_area_dates, norm_mean_area.values)
breaks_area = pwlf_fun_area.fit(n_segs)
breaks_area = [y for y in breaks_area if y != norm_mean_area_dates[0]]
breaks_area = [y for y in breaks_area if y != norm_mean_area_dates[-1]]
breaks_area = [round(y) for y in breaks_area]
area_fit = pwlf_fun_area.predict(YEARS)
area_rates = np.diff(area_fit) / np.diff(YEARS)

# norm_mean_length, norm_std_length = met.normChangeStats(all_glaciers, 'interplengths', startdate=first_full_year)
# norm_mean_length = norm_mean_length[~np.isnan(norm_mean_length)]
# norm_mean_length_dates = norm_mean_length.index.values
# pwlf_fun_length = pwlf.PiecewiseLinFit(norm_mean_length_dates, norm_mean_length.values)
# breaks_length = pwlf_fun_length.fit(n_segs)
# breaks_length = [y for y in breaks_length if y != norm_mean_length_dates[0]]
# breaks_length = [y for y in breaks_length if y != norm_mean_length_dates[-1]]
# breaks_length = [round(y) for y in breaks_length]
# length_fit = pwlf_fun_length.predict(YEARS)
# length_rates = np.diff(length_fit) / np.diff(YEARS)

print('Prior to {}, mean area (and length) change was {:.1f}% per decade.'.format(breaks_area, 10*100*area_rates[0]))
print('After {}, mean area (and length) change was {:.1f}% per decade.'.format(breaks_area, 10*100*area_rates[-1]))
print('This represents a {:.1f}x change in rate.'.format(area_rates[-1]/area_rates[0]))
# print('\nPrior to {}, mean length change was {:.1f}% per decade.'.format(breaks_length, 10*100*length_rates[0]))
# print('After {}, mean length change was {:.1f}% per decade.'.format(breaks_length, 10*100*length_rates[-1]))
# print('This represents a {:.1f}x change in rate.'.format(length_rates[-1]/length_rates[0]))


# Supplementary Table 2
Change GID and run to print values for each glacier ID in the table.

In [ ]:
gid = 90

significant_area = met.significantChange(all_glaciers, 'interpareas', startdate=first_full_year)
significant_length = met.significantChange(all_glaciers, 'interplengths', startdate=first_full_year)

print('Glacier ID: {}\n'.format(gid))
print('Observations: {}'.format(len(all_glaciers[gid].obsseries)))
print('Net area change: {:.2f}'.format(all_glaciers[gid].areas.diff().cumsum().iloc[-1]))
print('Area change is significant? {}'.format(significant_area.loc[gid]))
print('Net length change: {:.2f}'.format(all_glaciers[gid].lengths.diff().cumsum().iloc[-1]))
print('Length change is significant? {}'.format(significant_length.loc[gid]))
print('Breakpoint: {}'.format(met.getBreakPoints(all_glaciers[gid], 'interpareas', n_segs=2, startdate=first_full_year)[0][0]))


# Figure 6: MAR ice sheet climate variables

In [ ]:
# Load MAR data and resample, subset

mar_data = clm.loadData(mar_dir, dtype='mar')
mar = mar_data.resample(TIME='AS', keep_attrs=True).mean()
mar = clm.subsetGeographic(mar, ice_bounds, dtype='mar')
mar = clm.subsetIce(mar, threshold=ice_minimum)
mar = clm.subsetTopography(mar, topo_range, dtype='mar')

In [ ]:
# Plot annual and decadal MAR variables averaged across glacier coordinates

# -- PORTRAIT ORIENTATION
fig, axs = plt.subplots(nrows=4, ncols=2, sharex=True, figsize=(figwidth*2, figwidth*2.5))
axs = axs.flatten()

# -- Surface mass balance
mar.SMB.load()
cpl.marAnomalyBulk(axs[0], mar, glacier_coords, 'SMB', ann_mean=True, error=True, dec_mean=True)
axs[0].set_ylim(bottom=-1.5, top=1.5)
axs[0].xaxis.label.set_visible(False)
axs[0].annotate(text='(a)', xy=(-0.3, 1.05), xycoords='axes fraction')

# -- Meltwater production
mar.ME.load()
cpl.marAnomalyBulk(axs[2], mar, glacier_coords, 'ME', ann_mean=True, error=True, dec_mean=True)
axs[2].set_ylim(bottom=-1.5, top=1.5)
axs[2].xaxis.label.set_visible(False)
axs[2].annotate(text='(b)', xy=(-0.3, 1.05), xycoords='axes fraction')

# -- Runoff
mar.RU.load()
cpl.marAnomalyBulk(axs[3], mar, glacier_coords, 'RU', ann_mean=True, error=True, dec_mean=True)
axs[3].set_ylim(bottom=-1.5, top=1.5)
axs[3].xaxis.label.set_visible(False)
axs[3].annotate(text='(c)', xy=(-0.3, 1.05), xycoords='axes fraction')

# -- Number of melt days
ig, er, am, dm, dec_mean_medays = cpl.marMeltDaysAnomaly(axs[4], mar_data, glacier_coords, ann_mean=True, error=True, dec_mean=True)
axs[4].set_ylim(bottom=-40, top=40)
axs[4].xaxis.label.set_visible(False)
axs[4].annotate(text='(d)', xy=(-0.3, 1.05), xycoords='axes fraction')

# -- Number of runoff days
ig, er, am, dm, dec_mean_rudays = cpl.marRunoffDaysAnomaly(axs[5], mar_data, glacier_coords, ann_mean=True, error=True, dec_mean=True)
axs[5].set_ylim(bottom=-50, top=50)
axs[5].xaxis.label.set_visible(False)
axs[5].annotate(text='(e)', xy=(-0.3, 1.05), xycoords='axes fraction')

# -- Snowfall
mar.SF.load()
cpl.marAnomalyBulk(axs[6], mar, glacier_coords, 'SF', ann_mean=True, error=True, dec_mean=True)
axs[6].set_ylim(bottom=-.15, top=.2)
axs[6].annotate(text='(f)', xy=(-0.3, 1.05), xycoords='axes fraction')
axs[6].set_xlim(left='1969-01-01', right='2021-01-01')
axs[6].set_xticks(['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
axs[6].set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])

# -- Rainfall
mar.RF.load()
cpl.marAnomalyBulk(axs[7], mar, glacier_coords, 'RF', ann_mean=True, error=True, dec_mean=True)
axs[7].set_ylim(bottom=-.15, top=.2)
axs[7].annotate(text='(g)', xy=(-0.3, 1.05), xycoords='axes fraction')
axs[7].set_xlim(left='1969-01-01', right='2021-01-01')
axs[7].set_xticks(['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
axs[7].set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])

# -- Put legend in axs[1] location and turn off axis
fig.legend(handles=axs[7].get_legend_handles_labels()[0][-3:], ncol=1, loc='upper left', bbox_to_anchor=(0.6, 0.9))
axs[1].axis('off')
plt.tight_layout()
plt.savefig('{}fig6_mar.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}fig6_mar.png'.format(outpath), bbox_inches='tight', dpi=300)


# -- LANDSCAPE ORIENTATION
# fig, axs = plt.subplots(
#     nrows=2, ncols=4, 
#     sharex=True, sharey=False, 
#     figsize=(figwidth*4, figwidth*1.5))
# axs = axs.flatten()

# mar.SMB.load()
# ig, er, am, dm = cpl.marAnomalyBulk(axs[0], mar, glacier_coords, 'SMB', ann_mean=True, error=True, dec_mean=True, style=style)
# axs[0].set_ylim(bottom=-1.5, top=1.5)
# axs[0].xaxis.label.set_visible(False)
# axs[0].annotate(text='(a)', xy=(-0.3, 1.05), xycoords='axes fraction')

# ig, er, am, dm = cpl.marAnomalyBulk(axs[1], mar, glacier_coords, 'ME', ann_mean=True, error=True, dec_mean=True, style=style)
# axs[1].set_ylim(bottom=-1.5, top=1.5)
# axs[1].xaxis.label.set_visible(False)
# axs[1].annotate(text='(b)', xy=(-0.3, 1.05), xycoords='axes fraction')

# ig, er, am, dm, dec_mean_medays = cpl.marMeltDaysAnomaly(axs[5], mar_data, glacier_coords, ann_mean=True, error=True, dec_mean=True, style=style)
# axs[5].set_ylim(bottom=-40, top=40)
# axs[5].annotate(text='(c)', xy=(-0.3, 1.05), xycoords='axes fraction')

# ig, er, am, dm = cpl.marAnomalyBulk(axs[2], mar, glacier_coords, 'RU', ann_mean=True, error=True, dec_mean=True, style=style)
# axs[2].set_ylim(bottom=-1.5, top=1.5)
# axs[2].xaxis.label.set_visible(False)
# axs[2].annotate(text='(d)', xy=(-0.3, 1.05), xycoords='axes fraction')

# ig, er, am, dm, dec_mean_rudays = cpl.marRunoffDaysAnomaly(axs[6], mar_data, glacier_coords, ann_mean=True, error=True, dec_mean=True, style=style)
# axs[6].set_ylim(bottom=-50, top=50)
# axs[6].annotate(text='(e)', xy=(-0.3, 1.05), xycoords='axes fraction')

# ig, er, am, dm = cpl.marAnomalyBulk(axs[3], mar, glacier_coords, 'SF', ann_mean=True, error=True, dec_mean=True, style=style)
# axs[3].set_ylim(bottom=-.15, top=.2)
# axs[3].annotate(text='(f)', xy=(-0.3, 1.05), xycoords='axes fraction')
# axs[3].set_xlim(left='1969-01-01', right='2021-01-01')
# axs[3].xaxis.label.set_visible(False)
# axs[3].set_xticks(['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
# axs[3].set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])

# mar.RF.load()
# ig, er, am, dm = cpl.marAnomalyBulk(axs[7], mar, glacier_coords, 'RF', ann_mean=True, error=True, dec_mean=True, style=style)
# axs[7].set_ylim(bottom=-.15, top=.2)
# axs[7].annotate(text='(g)', xy=(-0.3, 1.05), xycoords='axes fraction')
# axs[7].set_xlim(left='1969-01-01', right='2021-01-01')
# axs[7].set_xticks(['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
# axs[7].set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])

# fig.legend(handles=[am, er, dm], ncol=1, loc='upper left', bbox_to_anchor=(0.05, 0.4))
# axs[4].axis('off')
# plt.tight_layout()




## Values for Fig. 6

Calculate bulk anomaly (across several glacier coordinates) and decadal mean anomaly for MAR data.

For dvar, choose from:
- 'SMB' (surface mass balance)
- 'ME' (meltwater production)
- 'RU' (runoff)
- 'SF' (snowfall)
- 'RF' (rainfall)

In [ ]:
dvar = 'ME'

mar[dvar].load()
bulk_mean = clm.bulkMeanMAR(mar, glacier_coords, dvar)
bulk_anom = clm.bulkAnomalyMAR(mar, glacier_coords, dvar)
dec_anom = clm.decadalAnomalyMAR(mar, glacier_coords, dvar)

dec_anom

In [ ]:
start_decade = '1990-01-01'
end_decade = '2000-01-01'

abs_change = dec_anom[end_decade] - dec_anom[start_decade]

dec_mean = dec_anom + bulk_mean
pct_change = 100 * (dec_mean[end_decade] - dec_mean[start_decade]) / dec_mean[start_decade]

print('Absolute change in {} between {} and {}: {:.3f} m'.format(dvar, start_decade, end_decade, abs_change))
print('Percent change in {} between {} and {}: {:.0f}%'.format(dvar, start_decade, end_decade, pct_change))

In [ ]:
start_decade = '2000-01-01'
end_decade = '2010-01-01'

mean_rudays = dec_mean_rudays + 113

abs_change = mean_rudays[end_decade] - mean_rudays[start_decade]
pct_change = 100 * (mean_rudays[end_decade] - mean_rudays[start_decade]) / mean_rudays[start_decade]

print('Absolute change in RU days between {} and {}: {:.0f}'.format(start_decade, end_decade, abs_change))
print('Percent change in RU days between {} and {}: {:.0f}%'.format(start_decade, end_decade, pct_change))

# Ocean data

In [ ]:
# Load Ocean Temperature Data

# -- ECCO
ecco_sst = clm.loadData(ecco_sst_dir, dtype='ecco5')
ecco_sst = clm.subsetGeographic(ecco_sst, ocean_bounds, dtype='ecco5')
ecco_sst = clm.subsetTime(ecco_sst, times, dtype='ecco5')
ecco_sst = ecco_sst.resample(time='AS', keep_attrs=True).mean()
ecco_sst = clm.eccoTemperature(ecco_sst, dtype='ecco5')
ecco_sst.temperature.load()

# -- ICES-DK
icesdk_data = clm.loadData(icesdk_file, dtype='icesdk')
icesdk_data = clm.subsetGeographic(icesdk_data, ocean_bounds, dtype='icesdk')
icesdk_data = clm.subsetTime(icesdk_data, times, dtype='icesdk')

# -- Hadley-OI
hadley_sst = clm.loadData(hadley_sst_dir, dtype='hadley')
hadley_sst = clm.subsetGeographic(hadley_sst, ocean_bounds, dtype='hadley')
hadley_sst = clm.subsetTime(hadley_sst, times, dtype='hadley')
hadley_sst = hadley_sst.resample(time='AS', keep_attrs=True).mean()
hadley_sst.SST.load()

# Figure 7: Ocean subsurface temperatures

In [ ]:
# Plot ocean temperature at specified depth at each point

fig, axs = plt.subplots(
    nrows=4, ncols=2, 
    sharex=True, sharey=True, 
    figsize=(figwidth*2, figwidth*2.5))
# fig, axs = plt.subplots(
#     nrows=2, ncols=4, 
#     sharex=True, sharey=True, 
#     figsize=(figwidth*4, figwidth*1.5))
axs = axs.flatten()
labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']

ecco_decadal = pd.DataFrame(index=['1990', '2000', '2010'],
                            columns=['0', '1', '2', '3', '4', '5', '6', '7'])
ices_decadal = pd.DataFrame(index=['1980', '1990', '2000', '2010'],
                            columns=['0'])

for coordinate in ocean_points:
    nax = axs[ocean_points.index(coordinate)]
    if ocean_points.index(coordinate) == 0:
        eag, edg, iag, idg, ecco_decade, ices_decade = cpl.subSurfaceTemperatureAnomaly(
            nax, depth=ocean_depth, coordinate=coordinate, 
            ecco=ecco_sst, ecco_dtype='ecco5', icesdk=icesdk_data, 
            dec_mean=True, spgrid=True, style=style)
        ices_decadal[str(ocean_points.index(coordinate))] = ices_decade.anomaly.values
    else:
        _, _, _, _, ecco_decade, _ = cpl.subSurfaceTemperatureAnomaly(
            nax, depth=ocean_depth, coordinate=coordinate, 
            ecco=ecco_sst, 
            dec_mean=True, spgrid=True, style=style)
    ecco_decadal[str(ocean_points.index(coordinate))] = ecco_decade.values
    nax.set_title('Point {}: {}'.format(
        ocean_points.index(coordinate)+1, 
        ocean_point_names[ocean_points.index(coordinate)]))
    nax.set_ylim(bottom=-1.25, top=1.45)
    nax.set_xlim(left='1969-01-01', right='2021-01-01')
    nax.set_xticks(
        ['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
    nax.set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])
    label = labels[ocean_points.index(coordinate)]
    nax.annotate(text=label, xy=(-0.15, 1.05), xycoords='axes fraction')

cpl.subplotGridLabels(
    fig, 
    title='Ocean temperature anomaly at {} m'.format(ocean_depth), 
    ylabel='Temperature anomaly ($^oC$)', 
    xlabel='Year')

# -- add legend for whole figure
fig.legend(handles=[eag, edg, iag, idg], ncol=2, loc='upper center', bbox_to_anchor=(0.5, 0.04))

plt.tight_layout()

plt.savefig('{}fig7_subsurfacetemp.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}fig7_subsurfacetemp.png'.format(outpath), bbox_inches='tight', dpi=300)


In [ ]:
print('Difference (*C) between 1990 and 2000 for ECCO:')
print(ecco_decadal.loc['2000'] - ecco_decadal.loc['1990'])
print('\nDifference (*C) between 2000 and 2010 for ECCO:')
print(ecco_decadal.loc['2010'] - ecco_decadal.loc['2000'])
ecco_decadal

In [ ]:
print('Difference (*C) between 2000 and 1990 for ICES:')
print(ices_decadal.loc['2000'] - ices_decadal.loc['1990'])
ices_decadal

# Figure 8: Sea surface temperature

In [ ]:
# Plot sea surface temperature at each point
fig, axs = plt.subplots(
    nrows=4, ncols=2, 
    sharex=True, sharey=True, 
    figsize=(figwidth*2, figwidth*2.5))
# fig, axs = plt.subplots(
#     nrows=2, ncols=4, 
#     sharex=True, sharey=True, 
#     figsize=(figwidth*4, figwidth*1.5))
axs = axs.flatten()
labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']

ecco_decadal = pd.DataFrame(index=['1990', '2000', '2010'],
                            columns=['0', '1', '2', '3', '4', '5', '6', '7'])
hadley_decadal = pd.DataFrame(index=['1970', '1980', '1990', '2000', '2010', '2020'],
                            columns=['0', '1', '2', '3', '4', '5', '6', '7'])

for coordinate in ocean_points:
    nax = axs[ocean_points.index(coordinate)]
    eag, edg, hag, hdg, ecco_decade, hadley_decade = cpl.seaSurfaceTemperatureAnomaly(
        nax, coordinate=coordinate, 
        ecco=ecco_sst, ecco_dtype='ecco5', hadley=hadley_sst, 
        dec_mean=True, spgrid=True, style=style)
    ecco_decadal[str(ocean_points.index(coordinate))] = ecco_decade.values
    hadley_decadal[str(ocean_points.index(coordinate))] = hadley_decade.values
    nax.set_title('Point {}: {}'.format(
        ocean_points.index(coordinate)+1, 
        ocean_point_names[ocean_points.index(coordinate)]))
    nax.set_ylim(bottom=-2.05, top=1.5)
    nax.set_xlim(left='1969-01-01', right='2021-01-01')
    nax.set_xticks(
        ['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
    nax.set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])
    label = labels[ocean_points.index(coordinate)]
    nax.annotate(text=label, xy=(-0.15, 1.05), xycoords='axes fraction')

cpl.subplotGridLabels(
    fig, 
    title='Sea surface temperature anomaly', 
    ylabel='Temperature anomaly ($^oC$)', 
    xlabel='Year')

# -- add legend for whole figure
fig.legend(handles=[eag, edg, hag, hdg], ncol=2, loc='upper center', bbox_to_anchor=(0.5, 0.04))

plt.tight_layout()

plt.savefig('{}fig8_seasurfacetemp.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}fig8_seasurfacetemp.png'.format(outpath), bbox_inches='tight', dpi=300)


In [ ]:
print('Difference (*C) between 1990 and 2000 for ECCO:')
print(ecco_decadal.loc['2000'] - ecco_decadal.loc['1990'])
print('\nDifference (*C) between 2000 and 2010 for ECCO:')
print(ecco_decadal.loc['2010'] - ecco_decadal.loc['2000'])
ecco_decadal

In [ ]:
print('Difference (*C) between 1990 and 2000 for Hadley-OI:')
print(hadley_decadal.loc['2000'] - hadley_decadal.loc['1990'])
print('\nDifference (*C) between 2000 and 2010 for Hadley-OI:')
print(hadley_decadal.loc['2010'] - hadley_decadal.loc['2000'])
hadley_decadal

In [ ]:
# Load Sea Ice Concentration Data

# -- Hadley-OI
hadley_sic = clm.loadData(hadley_sic_dir, dtype='hadley')
hadley_sic = clm.subsetGeographic(hadley_sic, ocean_bounds, dtype='hadley')
hadley_sic = clm.subsetTime(hadley_sic, times, dtype='hadley')
hadley_sic.SEAICE.load()

# -- NOAA
noaa_sic = clm.loadData(noaa_sic_dir, dtype='noaa')
noaa_sic = clm.subsetGeographic(noaa_sic, ocean_bounds, 'noaa')
noaa_sic = clm.subsetTime(noaa_sic, times, dtype='noaa')
noaa_sic[noaa_var].load()

# Figure 9: Sea ice season length

In [ ]:
# Get length of sea ice season at each point

fig, axs = plt.subplots(
    nrows=4, ncols=2, 
    sharex=True, sharey=True, 
    figsize=(figwidth*2, figwidth*2.5))
# fig, axs = plt.subplots(
#     nrows=2, ncols=4, 
#     sharex=True, sharey=True, 
#     figsize=(figwidth*4, figwidth*1.5))
axs = axs.flatten()
labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']

for coordinate in ocean_points:
    # Determine ice season length
    nax = axs[ocean_points.index(coordinate)]
    _, hag, hdg, nag, ndg = cpl.seaIceSeasonLength(
        nax, coordinate, sic_minimum, 
        hadley=hadley_sic, noaa=noaa_sic, noaa_var=noaa_var, 
        dec_mean=True, spgrid=True, style=style)
    nax.set_title('Point {}: {}'.format(
        ocean_points.index(coordinate)+1, 
        ocean_point_names[ocean_points.index(coordinate)]))
    nax.set_ylim(bottom=2.75, top=12.25)
    nax.set_xlim(left='1969-01-01', right='2021-01-01')
    nax.set_xticks(
        ['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
    nax.set_yticks([0,4,8,12])
    nax.set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])
    label = labels[ocean_points.index(coordinate)]
    nax.annotate(text=label, xy=(-0.15, 1.05), xycoords='axes fraction')
    
cpl.subplotGridLabels(
    fig, 
    title='Sea ice season length', 
    xlabel='Year', 
    ylabel='Months')

# -- add legend for whole figure
fig.legend(handles=[hag, hdg, nag, ndg], ncol=2, loc='upper center', bbox_to_anchor=(0.5, 0.04))

plt.tight_layout()

plt.savefig('{}fig9_seaiceseasonlength.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}fig9_seaiceseasonlength.png'.format(outpath), bbox_inches='tight', dpi=300)


# Supplementary Figures: Seasonal sea ice concentration

In [ ]:
# Plot seasonal sea ice concentration at each point

fig, axs = plt.subplots(
    nrows=4, ncols=2, 
    sharex=True, sharey=True, 
    figsize=(figwidth*2, figwidth*2.5))
axs = axs.flatten()
labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']

for coordinate in ocean_points:
    nax = axs[ocean_points.index(coordinate)]
    _, hag, hdg, nag, ndg = cpl.seaIceConcentration(
        nax, coordinate=coordinate, season_start=2, 
        hadley=hadley_sic, noaa=noaa_sic, noaa_var=noaa_var, 
        dec_mean=True, spgrid=True, style=style)
    nax.set_title('Point {}: {}'.format(
        ocean_points.index(coordinate)+1, 
        ocean_point_names[ocean_points.index(coordinate)]))
    nax.set_ylim(bottom=-5, top=105)
    nax.set_xlim(left='1969-01-01', right='2021-01-01')
    nax.set_xticks(
        ['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
    nax.set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])
    label = labels[ocean_points.index(coordinate)]
    nax.annotate(text=label, xy=(-0.15, 1.05), xycoords='axes fraction')
    
cpl.subplotGridLabels(
    fig, 
    title='FMA sea ice concentration', 
    xlabel='Time', 
    ylabel='% concentration')

# -- add legend for whole figure
fig.legend(handles=[hag, hdg, nag, ndg], ncol=2, loc='upper center', bbox_to_anchor=(0.5, 0.04))

plt.tight_layout()

plt.savefig('{}suppfig1_sic_fma.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}suppfig1_sic_fma.png'.format(outpath), bbox_inches='tight', dpi=300)


In [ ]:
# Plot seasonal sea ice concentration at each point

fig, axs = plt.subplots(
    nrows=4, ncols=2, 
    sharex=True, sharey=True, 
    figsize=(figwidth*2, figwidth*2.5))
axs = axs.flatten()
labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']

for coordinate in ocean_points:
    nax = axs[ocean_points.index(coordinate)]
    _, hag, hdg, nag, ndg = cpl.seaIceConcentration(
        nax, coordinate=coordinate, season_start=5, 
        hadley=hadley_sic, noaa=noaa_sic, noaa_var=noaa_var, 
        dec_mean=True, spgrid=True, style=style)
    nax.set_title('Point {}: {}'.format(
        ocean_points.index(coordinate)+1, 
        ocean_point_names[ocean_points.index(coordinate)]))
    nax.set_ylim(bottom=-5, top=105)
    nax.set_xlim(left='1969-01-01', right='2021-01-01')
    nax.set_xticks(
        ['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
    nax.set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])
    label = labels[ocean_points.index(coordinate)]
    nax.annotate(text=label, xy=(-0.15, 1.05), xycoords='axes fraction')
    
cpl.subplotGridLabels(
    fig, 
    title='MJJ sea ice concentration', 
    xlabel='Time', 
    ylabel='% concentration')

# -- add legend for whole figure
fig.legend(handles=[hag, hdg, nag, ndg], ncol=2, loc='upper center', bbox_to_anchor=(0.5, 0.04))

plt.tight_layout()

plt.savefig('{}suppfig2_sic_mjj.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}suppfig2_sic_mjj.png'.format(outpath), bbox_inches='tight', dpi=300)

In [ ]:
# Plot seasonal sea ice concentration at each point

fig, axs = plt.subplots(
    nrows=4, ncols=2, 
    sharex=True, sharey=True, 
    figsize=(figwidth*2, figwidth*2.5))
axs = axs.flatten()
labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']

for coordinate in ocean_points:
    nax = axs[ocean_points.index(coordinate)]
    _, hag, hdg, nag, ndg = cpl.seaIceConcentration(
        nax, coordinate=coordinate, season_start=8, 
        hadley=hadley_sic, noaa=noaa_sic, noaa_var=noaa_var, 
        dec_mean=True, spgrid=True, style=style)
    nax.set_title('Point {}: {}'.format(
        ocean_points.index(coordinate)+1, 
        ocean_point_names[ocean_points.index(coordinate)]))
    nax.set_ylim(bottom=-5, top=105)
    nax.set_xlim(left='1969-01-01', right='2021-01-01')
    nax.set_xticks(
        ['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
    nax.set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])
    label = labels[ocean_points.index(coordinate)]
    nax.annotate(text=label, xy=(-0.15, 1.05), xycoords='axes fraction')
    
cpl.subplotGridLabels(
    fig, 
    title='ASO sea ice concentration', 
    xlabel='Time', 
    ylabel='% concentration')

# -- add legend for whole figure
fig.legend(handles=[hag, hdg, nag, ndg], ncol=2, loc='upper center', bbox_to_anchor=(0.5, 0.04))

plt.tight_layout()

plt.savefig('{}suppfig3_sic_aso.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}suppfig3_sic_aso.png'.format(outpath), bbox_inches='tight', dpi=300)

In [ ]:
# Plot seasonal sea ice concentration at each point

fig, axs = plt.subplots(
    nrows=4, ncols=2, 
    sharex=True, sharey=True, 
    figsize=(figwidth*2, figwidth*2.5))
axs = axs.flatten()
labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']

for coordinate in ocean_points:
    nax = axs[ocean_points.index(coordinate)]
    _, hag, hdg, nag, ndg = cpl.seaIceConcentration(
        nax, coordinate=coordinate, season_start=11, 
        hadley=hadley_sic, noaa=noaa_sic, noaa_var=noaa_var, 
        dec_mean=True, spgrid=True, style=style)
    nax.set_title('Point {}: {}'.format(
        ocean_points.index(coordinate)+1, 
        ocean_point_names[ocean_points.index(coordinate)]))
    nax.set_ylim(bottom=-5, top=105)
    nax.set_xlim(left='1969-01-01', right='2021-01-01')
    nax.set_xticks(
        ['1970-01-01', '1980-01-01', '1990-01-01', '2000-01-01', '2010-01-01', '2020-01-01'])
    nax.set_xticklabels(['1970', '1980', '1990', '2000', '2010', '2020'])
    label = labels[ocean_points.index(coordinate)]
    nax.annotate(text=label, xy=(-0.15, 1.05), xycoords='axes fraction')
    
cpl.subplotGridLabels(
    fig, 
    title='NDJ sea ice concentration', 
    xlabel='Time', 
    ylabel='% concentration')

# -- add legend for whole figure
fig.legend(handles=[hag, hdg, nag, ndg], ncol=2, loc='upper center', bbox_to_anchor=(0.5, 0.04))

plt.tight_layout()

plt.savefig('{}suppfig4_sic_ndj.{}'.format(outpath, figformat), bbox_inches='tight', dpi=300)
plt.savefig('{}suppfig4_sic_ndj.png'.format(outpath), bbox_inches='tight', dpi=300)

# Discussion

Comparison with Howat and Eddy (2011)

In [ ]:
retreating_2000_2010 = met.retreatingGlaciers(all_glaciers, startdate='2000-01-01', enddate='2009-12-31')
stable_2000_2010 = met.stableGlaciers(all_glaciers, 'interplengths', startdate='2000-01-01', enddate='2009-12-31')
print('Between 2000 and 2010, {:.0f}% of glaciers retreated, {:.0f}% were stable.'.format(
    100 * len(retreating_2000_2010) / len(all_glaciers),
    100 * len(stable_2000_2010) / len(all_glaciers)))

retreating_2010_2020 = met.retreatingGlaciers(all_glaciers, startdate='2010-01-01', enddate='2019-12-31')
stable_2010_2020 = met.stableGlaciers(all_glaciers, 'interplengths', startdate='2010-01-01', enddate='2019-12-31')
print('Between 2010 and 2020, {:.0f}% of glaciers retreated, {:.0f}% were stable.'.format(
    100 * len(retreating_2010_2020) / len(all_glaciers),
    100 * len(stable_2010_2020) / len(all_glaciers)))

retreating_2000_2020 = met.retreatingGlaciers(all_glaciers, startdate='2000-01-01', enddate='2019-12-31')
stable_2000_2020 = met.stableGlaciers(all_glaciers, 'interplengths', startdate='2000-01-01', enddate='2019-12-31')
print('Between 2000 and 2020, {:.0f}% of glaciers retreated, {:.0f}% were stable.'.format(
    100 * len(retreating_2000_2020) / len(all_glaciers),
    100 * len(stable_2000_2020) / len(all_glaciers)))